# Add upstream, downstream and basin information to the dataframe

* Purpose of script: add contextual data to the dataframe. The script will sum the volumetric information of all upstream, downstream and basin parameters of the dataframe.  
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170915

In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2017M11D08 UTC 13:11


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
INPUT_VERSION = 17
OUTPUT_VERSION = 6

SCRIPT_NAME = "Y2017M09D15_RH_Add_Basin_Data_V01"

S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/%s/output/" %(SCRIPT_NAME)

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/%s/input" %(SCRIPT_NAME)
EC2_OUTPUT_PATH = "/volumes/data/%s/output" %(SCRIPT_NAME)

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V%0.2d.pkl" %(INPUT_VERSION)
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"

OUTPUT_FILENAME = "Y2017M09D15_RH_Add_Basin_Data_V%0.2d" %(OUTPUT_VERSION)


Note: There are two polygons with the same PFAF_ID (353020). This is caused by the fact that both poygons would otherwise cross the 180 degree meridian

In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [4]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [5]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V17.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/mergedZonalStatsEE_V17.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V17.csv to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/mergedZonalStatsEE_V17.csv


In [6]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [7]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [8]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)
inputLocationHydroBasins = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_HYDROBASINS)

In [9]:
df_ee = pd.read_pickle(inputLocationEE)

In [10]:
df_ee.index.names = ['PFAF_ID']

In [11]:
df_HydroBasins = pd.read_pickle(inputLocationHydroBasins)


In [12]:
df_complete = df_HydroBasins.merge(df_ee,how="left",left_index=True, right_index=True)

Note: There are two polygons with the same PFAF_ID (353020). This is caused by the fact that both poygons would otherwise cross the 180 degree meridian. 

In [13]:
df_complete = df_complete.drop_duplicates(subset='PFAF_ID', keep='first')

## Functions

In [14]:
def calculateTotalDemand(useType,temporalResolution,year,month):
    # This function will add Dom Ind IrrLinear and Livestock of all basins in the input list
    
    if temporalResolution == "year":
        keyTotal = "total_volume_Tot%s_%s_Y%0.4d" %(useType,temporalResolution,year)
    else:
        keyTotal = "total_volume_Tot%s_%s_Y%0.4dM%0.2d" %(useType,temporalResolution,year,month)
    
    # Create Column with zeros
    dfDemand[keyTotal] = 0
    for demandType in demandTypes:
        if demandType == "IrrLinear" and temporalResolution == "year":
            key = "total_volume_%s%s_%sY%0.4d" %(demandType,useType,temporalResolution,year)
        else:
            key = "total_volume_%s%s_%sY%0.4dM%0.2d" %(demandType,useType,temporalResolution,year,month)
        dfDemand[keyTotal] = dfDemand[keyTotal] + df_complete[key]
    return dfDemand   


# This functions can take only one argument because I map them over the pooler.
def addUpstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = pd.DataFrame(index=df_part_temp2.index)
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Upstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("total*"))
            df_upstream = df_upstream.add_prefix("upstream_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out
    
    
def addDownstream2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    df_out = pd.DataFrame(index=df_part_temp2.index)
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Downstream_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("total*"))
            df_upstream = df_upstream.add_prefix("downstream_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out


def addBasin2(listje):
    df_full_temp = df_complete.copy()
    df_part_temp = df_full_temp[df_full_temp.index.isin(listje)]
    df_part_temp2 = df_part_temp.copy()
    #df_out = df_part_temp2.copy()
    df_out = pd.DataFrame(index=df_part_temp2.index)
    
    i = 0
    for index, row in df_part_temp2.iterrows():
        i += 1
        print("i: ",i  ," index: ", index)
        try:
            upstreamCatchments = df_part_temp2.loc[index, "Basin_PFAF_IDs"]
            upstreamCatchments = ast.literal_eval(upstreamCatchments)
            df_upstream = df_full_temp[df_full_temp.index.isin(upstreamCatchments)]
            # selecting columns based on regular expression
            df_upstream = df_upstream.filter(regex=("total*"))
            df_upstream = df_upstream.add_prefix("basin_")
            sumSeries = df_upstream.sum(0)
            for key, value in sumSeries.iteritems():
                df_out.loc[index, key] = value
            df_out.loc[index, "errorCode"] = 0
        except:
            print("error")
            df_out.loc[index, "errorCode"] = 1
            pass
    
    return df_out

## Script

In [15]:
demandTypes = ["PDom","PInd","IrrLinear","PLiv"]
useTypes = ["WW","WN"]
temporalResolutions = ["year","month"]
years = [2014]

In [16]:
dfDemand = pd.DataFrame(index=df_complete.index)
for temporalResolution in temporalResolutions:
    for useType in useTypes:
        for year in years:
            if temporalResolution == "year":
                month = 12
                print(useType,temporalResolution,year,month)
                dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)
            else:
                for month in range(1,13):
                    print(useType,temporalResolution,year,month)
                    dfDemand = calculateTotalDemand(useType,temporalResolution,year,month)          

WW year 2014 12
WN year 2014 12
WW month 2014 1
WW month 2014 2
WW month 2014 3
WW month 2014 4
WW month 2014 5
WW month 2014 6
WW month 2014 7
WW month 2014 8
WW month 2014 9
WW month 2014 10
WW month 2014 11
WW month 2014 12
WN month 2014 1
WN month 2014 2
WN month 2014 3
WN month 2014 4
WN month 2014 5
WN month 2014 6
WN month 2014 7
WN month 2014 8
WN month 2014 9
WN month 2014 10
WN month 2014 11
WN month 2014 12


In [17]:
dfDemand.head()

total_volume_TotWW_year_Y2014  total_volume_TotWN_year_Y2014  \
PFAF_ID                                                                 
111011                    6.257741e+07                   2.915234e+07   
111012                    8.204290e+06                   3.388488e+06   
111013                    7.569537e+06                   3.111568e+06   
111014                    1.046668e+06                   5.024309e+05   
111015                    1.118431e+08                   4.668647e+07   

         total_volume_TotWW_month_Y2014M01  total_volume_TotWW_month_Y2014M02  \
PFAF_ID                                                                         
111011                        5.071952e+06                       5.039243e+06   
111012                        6.827566e+05                       6.829166e+05   
111013                        6.301277e+05                       6.302355e+05   
111014                        8.651350e+04                       8.664306e+04   
111015                        9.301079e+06                       9.303778e+06   

         total_volume_TotWW_month_Y2014M03  total_volume_TotWW_month_Y2014M04  \
PFAF_ID                                                                         
111011                        5.096070e+06                       5.268799e+06   
111012                        6.831846e+05                       6.836583e+05   
111013                        6.304261e+05                       6.307669e+05   
111014                        8.685163e+04                       8.721648e+04   
111015                        9.309980e+06                       9.319181e+06   

         total_volume_TotWW_month_Y2014M05  total_volume_TotWW_month_Y2014M06  \
PFAF_ID                                                                         
111011                        5.233163e+06                       5.282180e+06   
111012                        6.840569e+05                       6.843250e+05   
111013                        6.310496e+05                       6.312474e+05   
111014                        8.751861e+04                       8.770851e+04   
111015                        9.327492e+06                       9.333573e+06   

         total_volume_TotWW_month_Y2014M07  total_volume_TotWW_month_Y2014M08  \
PFAF_ID                                                                         
111011                        5.382082e+06                       5.472195e+06   
111012                        6.844042e+05                       6.844215e+05   
111013                        6.313145e+05                       6.313229e+05   
111014                        8.775453e+04                       8.777285e+04   
111015                        9.335649e+06                       9.335846e+06   

                       ...                  total_volume_TotWN_month_Y2014M03  \
PFAF_ID                ...                                                      
111011                 ...                                       2.372698e+06   
111012                 ...                                       2.818678e+05   
111013                 ...                                       2.589287e+05   
111014                 ...                                       4.149858e+04   
111015                 ...                                       3.881478e+06   

         total_volume_TotWN_month_Y2014M04  total_volume_TotWN_month_Y2014M05  \
PFAF_ID                                                                         
111011                        2.448676e+06                       2.443976e+06   
111012                        2.823415e+05                       2.827401e+05   
111013                        2.592695e+05                       2.595522e+05   
111014                        4.186342e+04                       4.216556e+04   
111015                        3.889610e+06                       3.896934e+06   

         total_volume_TotWN_month_Y2014M06  total_volume_TotWN_month_Y2014M07  \
PFAF_ID 

In [18]:
df_complete = df_complete.merge(dfDemand,how="left",left_index=True,right_index=True)

In [19]:
mp.cpu_count()

8

In [20]:
indices_full = df_complete.index.values
indices_split = np.array_split(indices_full, mp.cpu_count())

In [21]:
indices_split[1].shape

(2050,)

In [22]:
mp.log_to_stderr()

In [23]:
logger = mp.get_logger()
logger.setLevel(logging.INFO)

In [24]:
pool = mp.Pool(mp.cpu_count())

[INFO/ForkPoolWorker-1] child process calling self.run()
[INFO/ForkPoolWorker-2] child process calling self.run()
[INFO/ForkPoolWorker-3] child process calling self.run()
[INFO/ForkPoolWorker-4] child process calling self.run()
[INFO/ForkPoolWorker-5] child process calling self.run()
[INFO/ForkPoolWorker-6] child process calling self.run()
[INFO/ForkPoolWorker-7] child process calling self.run()
[INFO/ForkPoolWorker-8] child process calling self.run()


i:  1  index:  146660
i:  1  index:  561109
i:  1  index:  435045
i:  1  index:  634207
i:  1  index:  231609
i:  1  index:  111011
i:  1  index:  312977
i:  1  index:  742977
i:  2  index:  111012
i:  2  index:  146670
i:  2  index:  435046
i:  2  index:  561200
i:  2  index:  634208
i:  2  index:  312978
i:  2  index:  231700
i:  2  index:  742978
i:  3  index:  111013
i:  3  index:  146680
i:  3  index:  634209
i:  3  index:  435047
i:  3  index:  312979
i:  3  index:  231801
i:  3  index:  561210
i:  4  index:  146690
i:  3  index:  742979
i:  4  index:  111014
i:  4  index:  435048
i:  4  index:  634301
i:  4  index:  231802
i:  4  index:  312980
i:  4  index:  561220
i:  5  index:  146700
i:  5  index:  111015
i:  4  index:  742981
i:  5  index:  435049
i:  5  index:  634302
i:  5  index:  561230
i:  5  index:  231803
i:  5  index:  312990
i:  6  index:  146801
i:  6  index:  111016
i:  5  index:  742982
i:  6  index:  435050
i:  6  index:  634303
i:  6  index:  561240
i:  6  ind

i:  45  index:  751404
i:  45  index:  112013
i:  45  index:  313620
i:  45  index:  634706
i:  48  index:  151136
i:  46  index:  751405
i:  47  index:  232290
i:  46  index:  112014
i:  46  index:  436626
i:  47  index:  561507
i:  46  index:  313630
i:  46  index:  634707
i:  49  index:  151137
i:  47  index:  112015
i:  47  index:  751406
i:  48  index:  232301
i:  47  index:  436627
i:  48  index:  561508
i:  47  index:  634708
i:  50  index:  151138
i:  47  index:  313640
i:  49  index:  232302
i:  48  index:  436628
i:  48  index:  112016
i:  48  index:  751407
i:  49  index:  561509
i:  48  index:  634709
i:  49  index:  112017
i:  48  index:  313650
i:  51  index:  151139
i:  49  index:  751408
i:  50  index:  232303
i:  49  index:  436629
i:  50  index:  561601
i:  49  index:  313660
i:  52  index:  151140
i:  50  index:  112018
i:  50  index:  751409
i:  49  index:  634801
i:  51  index:  232304
i:  50  index:  436631
i:  51  index:  561602
i:  50  index:  313670
i:  53  ind

i:  89  index:  112801
i:  90  index:  436690
i:  90  index:  635180
i:  89  index:  752101
i:  89  index:  322190
i:  92  index:  561960
i:  93  index:  232810
i:  90  index:  112802
i:  91  index:  436691
i:  93  index:  151221
i:  90  index:  322211
i:  90  index:  752102
i:  91  index:  635190
i:  94  index:  232820
i:  93  index:  561970
i:  91  index:  112803
i:  92  index:  436692
i:  94  index:  151222
i:  91  index:  322212
i:  92  index:  635201
i:  91  index:  752103
i:  93  index:  436693
i:  92  index:  112804
i:  95  index:  151223
i:  95  index:  232830
i:  92  index:  322213
i:  93  index:  635202
i:  92  index:  752104
i:  94  index:  561981
i:  94  index:  436694
i:  93  index:  112805
i:  96  index:  232840
i:  93  index:  322214
i:  94  index:  635203
i:  95  index:  436695
i:  96  index:  151224
i:  93  index:  752105
i:  94  index:  112806
i:  97  index:  232850
i:  95  index:  561982
i:  95  index:  635204
i:  97  index:  151225
i:  96  index:  436696
i:  94  ind

i:  133  index:  114288
i:  133  index:  562384
i:  136  index:  234003
i:  134  index:  635607
i:  136  index:  151607
i:  132  index:  752164
i:  135  index:  437001
i:  131  index:  322310
i:  134  index:  114289
i:  134  index:  562385
i:  137  index:  151608
i:  137  index:  234004
i:  135  index:  635608
i:  132  index:  322320
i:  133  index:  752165
i:  136  index:  437002
i:  135  index:  114290
i:  138  index:  234005
i:  135  index:  562386
i:  138  index:  151609
i:  134  index:  752166
i:  136  index:  635609
i:  137  index:  437003
i:  136  index:  114300
i:  133  index:  322330
i:  136  index:  562387
i:  135  index:  752167
i:  137  index:  635700
i:  139  index:  151700
i:  138  index:  437004
i:  139  index:  234006
i:  137  index:  114410
i:  134  index:  322340
i:  140  index:  151801
i:  138  index:  635801
i:  136  index:  752168
i:  139  index:  437005
i:  137  index:  562388
i:  140  index:  234007
i:  138  index:  114411
i:  135  index:  322350
i:  141  index: 

...

i:  1889  index:  742667
i:  1909  index:  312623
i:  1898  index:  631067
i:  1902  index:  144506
i:  1907  index:  531108
i:  1907  index:  226090
i:  1894  index:  912577
i:  1899  index:  434426
i:  1890  index:  742668
i:  1910  index:  312624
i:  1899  index:  631068
i:  1908  index:  531109
i:  1895  index:  912578
i:  1903  index:  144507
i:  1900  index:  434427
i:  1908  index:  227101
i:  1911  index:  312625
i:  1891  index:  742669
i:  1900  index:  631069
i:  1896  index:  912580
i:  1901  index:  434428
i:  1904  index:  144508
i:  1909  index:  531200
i:  1909  index:  227102
i:  1912  index:  312626
i:  1892  index:  742671
i:  1901  index:  631070
i:  1897  index:  912590
i:  1902  index:  434429
i:  1905  index:  144509
i:  1910  index:  531201
i:  1913  index:  312627
i:  1910  index:  227103
i:  1893  index:  742672
i:  1902  index:  631081
i:  1903  index:  434430
i:  1906  index:  144600
i:  1898  index:  912600
i:  1911  index:  531202
i:  1914  index:  312628


i:  1948  index:  531592
i:  1938  index:  632300
i:  1951  index:  312686
i:  1942  index:  145208
i:  1931  index:  742822
i:  1949  index:  227389
i:  1934  index:  913442
i:  1940  index:  434696
i:  1939  index:  632401
i:  1949  index:  531593
i:  1943  index:  145209
i:  1932  index:  742823
i:  1950  index:  227390
i:  1952  index:  312687
i:  1935  index:  913443
i:  1940  index:  632402
i:  1941  index:  434697
i:  1950  index:  531594
i:  1933  index:  742824
i:  1944  index:  145300
i:  1951  index:  227401
i:  1953  index:  312688
i:  1936  index:  913444
i:  1942  index:  434698
i:  1945  index:  145401
i:  1941  index:  632403
i:  1951  index:  531595
i:  1934  index:  742825
i:  1952  index:  227402
i:  1937  index:  913445
i:  1954  index:  312689
i:  1943  index:  434699
i:  1946  index:  145402
i:  1935  index:  742826
i:  1952  index:  531596
i:  1953  index:  227403
i:  1942  index:  632404
i:  1955  index:  312691
i:  1938  index:  913446
i:  1944  index:  434700


i:  1976  index:  913604
i:  1972  index:  742885
i:  1992  index:  312860
i:  1990  index:  227804
i:  1978  index:  633037
i:  1984  index:  145777
i:  1988  index:  531905
i:  1977  index:  913605
i:  1973  index:  742886
i:  1981  index:  434923
i:  1991  index:  227805
i:  1985  index:  145778
i:  1979  index:  633038
i:  1989  index:  531906
i:  1993  index:  312870
i:  1982  index:  434924
i:  1974  index:  742887
i:  1980  index:  633039
i:  1978  index:  913606
i:  1990  index:  531907
i:  1986  index:  145779
i:  1992  index:  227806
i:  1994  index:  312880
i:  1981  index:  633041
i:  1979  index:  913607
i:  1991  index:  531908
i:  1983  index:  434925
i:  1995  index:  312890
i:  1975  index:  742888
i:  1987  index:  145780
i:  1993  index:  227807
i:  1982  index:  633042
i:  1992  index:  531909
i:  1980  index:  913608
i:  1994  index:  227808
i:  1988  index:  145791
i:  1984  index:  434926
i:  1976  index:  742889
i:  1996  index:  312901
i:  1983  index:  633043


i:  2022  index:  434977
i:  2020  index:  634112
i:  2030  index:  231208
i:  2030  index:  552000
i:  2031  index:  312947
i:  2026  index:  146310
i:  2013  index:  742939
i:  2017  index:  914407
i:  2021  index:  634113
i:  2023  index:  434978
i:  2031  index:  231209
i:  2032  index:  312948
i:  2031  index:  553000
i:  2027  index:  146320
i:  2014  index:  742940
i:  2018  index:  914408
i:  2022  index:  634114
i:  2032  index:  231300
i:  2032  index:  554000
i:  2024  index:  434979
i:  2033  index:  312949
i:  2015  index:  742941
i:  2028  index:  146330
i:  2019  index:  914409
i:  2023  index:  634115
i:  2033  index:  231401
i:  2033  index:  555000
i:  2016  index:  742942
i:  2034  index:  312950
i:  2025  index:  434981
i:  2029  index:  146340
i:  2020  index:  914501
i:  2034  index:  231402
i:  2034  index:  556000
i:  2024  index:  634116
i:  2026  index:  434982
i:  2035  index:  312960
i:  2021  index:  914502
i:  2017  index:  742943
i:  2030  index:  146350


[INFO/ForkPoolWorker-1] process shutting down
[INFO/ForkPoolWorker-5] process shutting down
[INFO/ForkPoolWorker-8] process shutting down
[INFO/ForkPoolWorker-2] process shutting down
[INFO/ForkPoolWorker-6] process shutting down
[INFO/ForkPoolWorker-4] process shutting down
[INFO/ForkPoolWorker-1] process exiting with exitcode 0
[INFO/ForkPoolWorker-2] process exiting with exitcode 0
[INFO/ForkPoolWorker-8] process exiting with exitcode 0
[INFO/ForkPoolWorker-7] process shutting down
[INFO/ForkPoolWorker-4] process exiting with exitcode 0
[INFO/ForkPoolWorker-6] process exiting with exitcode 0
[INFO/ForkPoolWorker-5] process exiting with exitcode 0
[INFO/ForkPoolWorker-3] process shutting down
[INFO/ForkPoolWorker-7] process exiting with exitcode 0
[INFO/ForkPoolWorker-3] process exiting with exitcode 0


In [25]:
df_upstream = pd.concat(pool.map(addUpstream2, indices_split))

In [26]:
df_downstream = pd.concat(pool.map(addDownstream2, indices_split))

In [27]:
df_basin = pd.concat(pool.map(addBasin2, indices_split))

In [28]:
pool.close()

In [29]:
df_complete = df_complete.merge(df_upstream,how="left",left_index=True,right_index=True)

In [30]:
df_complete = df_complete.merge(df_downstream,how="left",left_index=True,right_index=True)

In [31]:
df_complete = df_complete.merge(df_basin,how="left",left_index=True,right_index=True)

In [32]:
df_complete.to_pickle(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".pkl"))

In [33]:
df_complete.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILENAME+".csv"))

In [34]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V06.pkl to s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V06.pkl
upload: ../../../../data/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V06.csv to s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V06.csv


In [35]:
df_complete.head()

HYBAS_ID2  Unnamed: 0    HYBAS_ID  NEXT_DOWN   NEXT_SINK  \
PFAF_ID                                                              
111011   1060000010        6675  1060000010          0  1060000010   
111012   1060000100        6676  1060000100          0  1060000100   
111013   1060000110        6677  1060000110          0  1060000110   
111014   1060000150        6678  1060000150          0  1060000150   
111015   1060000160        6679  1060000160          0  1060000160   

           MAIN_BAS  DIST_SINK  DIST_MAIN  SUB_AREA  UP_AREA    ...      \
PFAF_ID                                                         ...       
111011   1060000010        0.0        0.0    1890.8   1890.8    ...       
111012   1060000100        0.0        0.0    2925.9   2925.9    ...       
111013   1060000110        0.0        0.0     893.5    893.5    ...       
111014   1060000150        0.0        0.0    4217.3   4217.4    ...       
111015   1060000160        0.0        0.0   16638.1  16638.1    ...       

         basin_total_volume_TotWN_month_Y2014M04  \
PFAF_ID                                            
111011                              2.448676e+06   
111012                              2.823415e+05   
111013                              2.592695e+05   
111014                              4.186342e+04   
111015                              3.889610e+06   

         basin_total_volume_TotWN_month_Y2014M05  \
PFAF_ID                                            
111011                              2.443976e+06   
111012                              2.827401e+05   
111013                              2.595522e+05   
111014                              4.216556e+04   
111015                              3.896934e+06   

         basin_total_volume_TotWN_month_Y2014M06  \
PFAF_ID                                            
111011                              2.468962e+06   
111012                              2.830082e+05   
111013                              2.597500e+05   
111014                              4.235545e+04   
111015                              3.902287e+06   

         basin_total_volume_TotWN_month_Y2014M07  \
PFAF_ID                                            
111011                              2.509266e+06   
111012                              2.830874e+05   
111013                              2.598171e+05   
111014                              4.240148e+04   
111015                              3.904112e+06   

         basin_total_volume_TotWN_month_Y2014M08  \
PFAF_ID                                            
111011                              2.543447e+06   
111012                              2.831047e+05   
111013                              2.598255e+05   
111014                              4.241980e+04   
111015                              3.904281e+06   

        basin_total_volume_TotWN_month_Y2014M09  \
PFAF_ID                                           
111011                             2.498844e+06   
111012                             2.829387e+05   
111013                             2.597075e+05   
111014                             4.227204e+04   
111015                             3.900583e+06   

        basin_total_volume_TotWN_month_Y2014M10  \
PFAF_ID                                           
111011                             2.432985e+06   
111012                             2.826311e+05   
111013                             2.594838e+05   
111014                             4.204755e+04   
111015                             3.894476e+06   

        basin_total_volume_TotWN_month_Y2014M11  \
PFAF_ID                                           
111011                             2.380002e+06   
111012                             2.820958e+05   
111013                             2.591036e+05   
111014                             4.165468e+04   
111015                             3.886032e+06   

        basin_total_volume_TotWN_month_Y2014M12  errorCode  
PFAF_ID                          

In [36]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:05:34.885566
